In [1]:
import gensim
from gensim import matutils, corpora
from gensim.models.ldamodel import LdaModel
import pandas as pd
import nltk
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
import pyLDAvis
import pyLDAvis.gensim
import numpy as np 
import snowball

In [2]:
tweet_data = pd.read_csv("CancerReport-clean-all-data-en.txt", delimiter="\t")

/anaconda/lib/python3.4/site-packages/IPython/core/interactiveshell.py:2902: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
tweet_data.shape

(40877, 40)

In [4]:
tweet_data.head()

,tweet_id,tweet_text,tweet_created_at,in_reply_to_status_id_str,in_reply_to_screen_name,retweet_count,favorite_count,machine_translated_language,geo_lat,geo_long,...,retweet_created_at,retweet_screen_name,retweet_user_created_at,retweet_person_name,retweet_statuses_count,retweet_friends_count,retweet_followers_count,retweet_urls,retweet_hashtag_text,retweet_usermentions_screen_name
0,185077342425714688,RT @jockosports: 2 more days until my #colonos...,2012-03-28 14:54:11,NaN,NaN,NaN,NaN,NaN,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,185081180222263297,My arm is so sore :( had a HPV injection at sc...,2012-03-28 15:09:26,NaN,NaN,NaN,NaN,NaN,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,185096288109461507,MsRhea Mike HEALTHY Wednesday Awareness mak...,2012-03-28 16:09:28,NaN,NaN,NaN,NaN,NaN,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,185108334121193472,@xomorgyyxo I asked him three times if it was ...,2012-03-28 16:57:20,NaN,NaN,NaN,NaN,NaN,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,185117800669523968,I have been poked and prodded today.and my boo...,2012-03-28 17:34:57,NaN,NaN,NaN,NaN,NaN,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
primary_tweets = tweet_data[tweet_data["retweet"] == False]

In [6]:
tweets = tweet_data["tweet_text"].tolist()

In [7]:
tweets[0]

'RT @jockosports: 2 more days until my #colonoscopy. I hope to raise awareness of colon cancer and prevention.  Follow my journey on #402 ...'

In [8]:
# this will be *all* tweets; independent of the hashtag
toked_tweets, kept_indices = snowball.build_gensim_corpus(tweets)

In [9]:
print(list(toked_tweets[0]))

['rt', 'days', 'colonoscopy', 'hope', 'raise', 'awareness', 'colon', 'cancer', 'prevention', 'follow', 'journey']


In [10]:
retweet_counts = np.nan_to_num(tweet_data["retweet_count"].tolist()) # note that I am assuming NaN == 0

In [11]:
retweet_counts[:10]

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [12]:
dictionary = corpora.Dictionary(toked_tweets)

In [13]:
dictionary.save("snowball.tweets.dict")

In [14]:
print(dictionary)

Dictionary(8890 unique tokens: ['des', 'selected', 'theory', 'whatever', 'organic']...)


In [15]:
gensim_corpus = [dictionary.doc2bow(tweet) for tweet in toked_tweets]#[[dictionary.token2id[t] for t in tweet] for tweet in toked_tweets]

In [16]:
lda = LdaModel(gensim_corpus, num_topics=20,
                    passes=10, alpha=0.001, id2word=dictionary)

In [17]:
lda.save("snowball-LDA-20-topics.model")

In [18]:
lda.show_topics()

['0.128*hpv + 0.123*co + 0.122*http + 0.061*vaccine + 0.039*rt + 0.026*gardasil + 0.018*girls + 0.014*vaccination + 0.014*vaccines + 0.011*study',
 '0.099*mcrcomeback + 0.070*gt + 0.037*imusicrcti + 0.033*control + 0.032*birth + 0.032*imusic_rcti + 0.028*kotakmisteriimusic + 0.021*rt + 0.018*didn + 0.018*abortion',
 '0.037*million + 0.034*prostate + 0.030*amp + 0.024*http + 0.021*co + 0.020*medical + 0.017*healthy + 0.016*prevention + 0.014*side + 0.013*via',
 '0.042*colonoscopy + 0.035*rt + 0.028*buying + 0.023*president + 0.021*clinics + 0.020*un + 0.015*use + 0.014*perform + 0.013*match + 0.013*lead',
 '0.055*de + 0.036*rt + 0.033*la + 0.024*est + 0.022*young + 0.021*healthcare + 0.021*treatments + 0.017*ppfa + 0.016*std + 0.014*deaths',
 '0.064*rt + 0.059*screenings + 0.048*cancer + 0.041*amp + 0.041*access + 0.032*care + 0.028*mammograms + 0.028*exams + 0.020*health + 0.015*thanks',
 '0.078*co + 0.078*rt + 0.074*http + 0.043*ecrchat + 0.025*phdchat + 0.025*reminder + 0.018*et + 0.

In [19]:
for_viz = pyLDAvis.gensim.prepare(lda, gensim_corpus, dictionary)

/anaconda/lib/python3.4/site-packages/pyLDAvis-1.2.0-py3.4.egg/pyLDAvis/_prepare.py:283: FutureWarning: order is deprecated, use sort_values(...)
  topic_proportion = (topic_freq / topic_freq.sum()).order(ascending=False)
/anaconda/lib/python3.4/site-packages/pyLDAvis-1.2.0-py3.4.egg/pyLDAvis/_prepare.py:154: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  sort('saliency', ascending=False). \
/anaconda/lib/python3.4/site-packages/pyLDAvis-1.2.0-py3.4.egg/pyLDAvis/_prepare.py:203: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  return token_table.sort(['Term', 'Topic'])


In [20]:
pyLDAvis.display(for_viz)

In [21]:
toked_tweets_by_tag, tags_to_raw_tweets = snowball.build_gensim_corpus(tweets, split_up_by_tag=True)

In [22]:
toked_tweets_by_tag.keys()

dict_keys(['#screened', 'mammogram', 'cancer prevention', '#GoingToTheDoctor', 'vaxx', '#vaccinated', '#fightcancer', 'Gardasil', 'colonoscopy', '#WomensHealth', 'pap smear', 'HPV', '#stopcancerb4itstarts', 'cervical cancer', 'other', 'HPV vaccination'])

In [23]:
import importlib; importlib.reload(snowball)

<module 'snowball' from '/Users/byron/dev/snowball/data/snowball.py'>

In [24]:
lda_cervical, corpus_cervical, dict_cervical= snowball.gen_lda_model(toked_tweets_by_tag['cervical cancer'])

In [25]:
for_viz_cervical = pyLDAvis.gensim.prepare(lda_cervical, corpus_cervical, dict_cervical)

/anaconda/lib/python3.4/site-packages/pyLDAvis-1.2.0-py3.4.egg/pyLDAvis/_prepare.py:283: FutureWarning: order is deprecated, use sort_values(...)
  topic_proportion = (topic_freq / topic_freq.sum()).order(ascending=False)
/anaconda/lib/python3.4/site-packages/pyLDAvis-1.2.0-py3.4.egg/pyLDAvis/_prepare.py:154: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  sort('saliency', ascending=False). \
/anaconda/lib/python3.4/site-packages/pyLDAvis-1.2.0-py3.4.egg/pyLDAvis/_prepare.py:203: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  return token_table.sort(['Term', 'Topic'])


In [26]:
pyLDAvis.display(for_viz_cervical)

In [27]:
### 
# split by arm
###
cervical_arm_tags = ["pap smear", "pap test", "HPV", "human papillomavirus","HPV vaccination","Gardasil","cervical cancer"]
comparison_study_arm_tags = ["#GoingToTheDoctor", "#WomensHealth", "colonoscopy","cancer prevention","cancer screening","mammogram","vaxx","#fightcancer","#stopcancerb4itstarts","#screened","#vaccinated"]

# get tweets corresponding to each 
cervical_arm_tweets = []
for tag in cervical_arm_tags: 
    cervical_arm_tweets.extend(toked_tweets_by_tag[tag])

comparison_arm_tweets = []
for tag in comparison_study_arm_tags:
    comparison_arm_tweets.extend(toked_tweets_by_tag[tag])

print("num cervical arm: %s; num comparison arm: %s" % (len(cervical_arm_tweets), len(comparison_arm_tweets)))



num cervical arm: 15359; num comparison arm: 19090


In [28]:
# stratified LDA
lda_cervical_arm, corpus_cervical_arm, dict_cervical_arm = snowball.gen_lda_model(cervical_arm_tweets)
for_viz_cervical = pyLDAvis.gensim.prepare(lda_cervical, corpus_cervical, dict_cervical)
pyLDAvis.display(for_viz_cervical)

/anaconda/lib/python3.4/site-packages/pyLDAvis-1.2.0-py3.4.egg/pyLDAvis/_prepare.py:283: FutureWarning: order is deprecated, use sort_values(...)
  topic_proportion = (topic_freq / topic_freq.sum()).order(ascending=False)
/anaconda/lib/python3.4/site-packages/pyLDAvis-1.2.0-py3.4.egg/pyLDAvis/_prepare.py:154: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  sort('saliency', ascending=False). \
/anaconda/lib/python3.4/site-packages/pyLDAvis-1.2.0-py3.4.egg/pyLDAvis/_prepare.py:203: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  return token_table.sort(['Term', 'Topic'])


In [29]:
lda_comp_arm, corpus_comp_arm, dict_comp_arm = snowball.gen_lda_model(comparison_arm_tweets)
for_viz_comp = pyLDAvis.gensim.prepare(lda_comp_arm, corpus_comp_arm, dict_comp_arm)
pyLDAvis.display(for_viz_comp)

/anaconda/lib/python3.4/site-packages/pyLDAvis-1.2.0-py3.4.egg/pyLDAvis/_prepare.py:283: FutureWarning: order is deprecated, use sort_values(...)
  topic_proportion = (topic_freq / topic_freq.sum()).order(ascending=False)
/anaconda/lib/python3.4/site-packages/pyLDAvis-1.2.0-py3.4.egg/pyLDAvis/_prepare.py:154: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  sort('saliency', ascending=False). \
/anaconda/lib/python3.4/site-packages/pyLDAvis-1.2.0-py3.4.egg/pyLDAvis/_prepare.py:203: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  return token_table.sort(['Term', 'Topic'])
